In [3]:
import pandas as pd 
import numpy as np 
import time 
import turicreate as  tc 
from sklearn.model_selection import train_test_split 

In [9]:
df = pd.read_excel('filtered_retail.xlsx', index_col = 0)
pre_len = len(df)
df.head()

,Date,Customer_ID,Transaction_ID,SKU_Category_x,SKU,Quantity,Sales_Amount,SKU_Category_y,Menu_Category,Item_ID,Sex,Weather,Zip_cd
0,2016-01-02,3686,3,0H2,CZUZX,1.0,6.35,0H2,Fruit Series,FS1,male,windy,70573
1,2016-01-02,1253,8,0H2,9STQJ,1.0,8.25,0H2,Fruit Series,FS3,female,sunny,70577
2,2016-01-02,7548,10,N8U,UNJKW,1.0,2.11,N8U,Signature Series,SS1,male,sunny,70562
3,2016-01-02,6044,12,N8U,EMJ1S,1.0,3.62,N8U,Signature Series,SS6,male,sunny,70566
4,2016-01-02,592,13,P42,B2IW9,1.0,9.00,P42,Fresh Milk,FM9,male,sunny,70571


In [10]:
# Quantity will be the number of items be purchased in each transaction
# filter out float number
df = df[df.Quantity>=1]
df.Quantity = df.Quantity.astype(int)
post_len = len(df)
print('{} Row is deleted because of the float number in Quantity column'
      .format(pre_len-post_len))

10 Row is deleted because of the float number in Quantity column


In [11]:
# review how many customerID in the transaction data
C_ID_list = df.Customer_ID.value_counts()
total_ID_nb = len(C_ID_list)
ID_nb_smaller_2 = len(np.where(C_ID_list<=2)[0])
print('There are {} unique ID in the transaction data'.format(total_ID_nb))
print('There are {} ID order equal or less than twice'.format(ID_nb_smaller_2))

There are 11916 unique ID in the transaction data
There are 8276 ID order equal or less than twice


In [12]:
aggregated_table = df.groupby(['Customer_ID', 'Zip_cd', 'Weather', 'Menu_Category']).sum().drop(['Transaction_ID', 'Sales_Amount'], axis=1).reset_index()
aggregated_table

,Customer_ID,Zip_cd,Weather,Menu_Category,Quantity
0,4,70576,windy,Egg Waffle,1
1,5,70572,windy,Fresh Milk,3
2,5,70573,sunny,Fresh Milk,4
3,5,70574,sunny,Fresh Milk,2
4,5,70577,sunny,Fresh Milk,4
...,...,...,...,...,...
32236,22615,70579,windy,Small Bites,1
32237,22616,70572,windy,Fresh Milk,1
32238,22621,70563,cloudy,Dessert Combo,2
32239,22621,70563,windy,Dessert Combo,1


In [14]:
# the model refer: https://apple.github.io/turicreate/docs/api/generated/turicreate.recommender.util.compare_models.html#turicreate.recommender.util.compare_models
# and https://medium.datadriveninvestor.com/how-to-build-a-recommendation-system-for-purchase-data-step-by-step-d6d7a78800b6
def recommender(data, algo, user_id, item_id, target, n_recom, n_show):
    if algo == 'popularity':
        model = tc.popularity_recommender.create(data, 
                                                 user_id=user_id, 
                                                 item_id=item_id, 
                                                 target=target)
    elif algo == 'cosine':
        model = tc.item_similarity_recommender.create(data, 
                                                      user_id=user_id, 
                                                      item_id=item_id, 
                                                      target=target, 
                                                      similarity_type='cosine')
    elif algo == 'pearson':
        model = tc.item_similarity_recommender.create(data, 
                                                      user_id=user_id, 
                                                      item_id=item_id, 
                                                      target=target, 
                                                      similarity_type='pearson')
        
    recom = model.recommend(k=n_recom)
#     recom.print_rows(n_show)
    return model, recom

In [51]:
tmp.head()

,Customer_ID,Item_ID,Quantity
0,4,EW6,1
1,5,FM2,4
2,11,TP7,1
3,13,TP9,1
4,14,F1,1


In [56]:
input_col = 'Customer_ID'
output_col = 'Item_ID'

tmp = df.groupby([input_col, output_col]).size().reset_index(name = 'Quantity')
df_matrix = pd.pivot_table(tmp, values = 'Quantity', index = input_col, columns = output_col)
df_matrix_norm = (df_matrix - df_matrix.min()) / (df_matrix.max() - df_matrix.min())

d = df_matrix_norm.reset_index() 
d.index.names = ['scaled_Quantity'] 
data_norm = pd.melt(d, id_vars = [input_col], value_name = 'scaled_Quantity').dropna()

train, test = train_test_split(data_norm, test_size = 0.2, random_state = 2022)

def setup_recomm(data, input_col, method, n_r = 1, n_s = 10):
    return recommender(tc.SFrame(data), method, input_col, output_col, 'scaled_Quantity', n_r, n_s)

pop1, pop2 = setup_recomm(train, input_col, 'popularity')
cos1, cos2 = setup_recomm(train, input_col, 'cosine')
ps1, ps2 = setup_recomm(train, input_col, 'pearson')

Preparing data set.

Data has 20408 observations with 10546 users and 147 items.

Data prepared in: 0.047736s

20408 observations to process; with 147 unique items.

recommendations finished on 1000/10546 queries. users per second: 400000

...

recommendations finished on 6000/10546 queries. users per second: 620732

recommendations finished on 7000/10546 queries. users per second: 629553

recommendations finished on 8000/10546 queries. users per second: 631463

recommendations finished on 9000/10546 queries. users per second: 638343

recommendations finished on 10000/10546 queries. users per second: 626488

In [57]:
models = [pop1, cos1, ps1]
tc.recommender.util.compare_models(tc.SFrame(test), models, model_names = ['popularity', 'cosine', 'pearson'])

PROGRESS: Evaluate model popularity


recommendations finished on 1000/4086 queries. users per second: 71017.7

recommendations finished on 2000/4086 queries. users per second: 64815.1

recommendations finished on 3000/4086 queries. users per second: 64318.3

recommendations finished on 4000/4086 queries. users per second: 62771.7

...

recommendations finished on 2000/4086 queries. users per second: 86214.3

recommendations finished on 3000/4086 queries. users per second: 81475.2

recommendations finished on 4000/4086 queries. users per second: 75356.5


Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    |  0.002936857562408223 | 0.0022026431718061663 |
|   2    | 0.0023250122369065113 | 0.0037934410181106218 |
|   3    | 0.0027736988089411005 |  0.006118453255017128 |
|   4    |  0.003059226627508566 |  0.008941099689998358 |
|   5    |  0.002936857562408223 |  0.010735845978136734 |
|   6    |  0.002977647250775001 |  0.013191385217816938 |
|   7    |  0.002692119432207537 |  0.013762440854951874 |
|   8    | 0.0025391581008321084 |  0.014884157285038332 |
|   9    | 0.0024745744275847014 |   0.016434165442976   |
|   10   |  0.002373959862946645 |  0.017657856093979466 |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.15636881559735963

Per User RMSE (best)
+-------------+-----------------------+----

[{'precision_recall_by_user': Columns:
  	Customer_ID	int
  	cutoff	int
  	precision	float
  	recall	float
  	count	int
  
  Rows: 73548
  
  Data:
  +-------------+--------+-----------+--------+-------+
  | Customer_ID | cutoff | precision | recall | count |
  +-------------+--------+-----------+--------+-------+
  |    15523    |   1    |    0.0    |  0.0   |   2   |
  |    15523    |   2    |    0.0    |  0.0   |   2   |
  |    15523    |   3    |    0.0    |  0.0   |   2   |
  |    15523    |   4    |    0.0    |  0.0   |   2   |
  |    15523    |   5    |    0.0    |  0.0   |   2   |
  |    15523    |   6    |    0.0    |  0.0   |   2   |
  |    15523    |   7    |    0.0    |  0.0   |   2   |
  |    15523    |   8    |    0.0    |  0.0   |   2   |
  |    15523    |   9    |    0.0    |  0.0   |   2   |
  |    15523    |   10   |    0.0    |  0.0   |   2   |
  +-------------+--------+-----------+--------+-------+
  [73548 rows x 5 columns]
  Note: Only the head of the SFrame is pr

In [45]:
ps1.recommend(k = 5).print_rows(25)

recommendations finished on 1000/10529 queries. users per second: 195198

recommendations finished on 2000/10529 queries. users per second: 240964

recommendations finished on 3000/10529 queries. users per second: 249480

recommendations finished on 4000/10529 queries. users per second: 275292

recommendations finished on 5000/10529 queries. users per second: 282534

...

recommendations finished on 6000/10529 queries. users per second: 274449

recommendations finished on 7000/10529 queries. users per second: 270929

recommendations finished on 8000/10529 queries. users per second: 278600

recommendations finished on 9000/10529 queries. users per second: 285914

recommendations finished on 10000/10529 queries. users per second: 288434

In [50]:
df.Item_ID.value_counts().sort_values(ascending = False).head(20)

SS1    2007
SS2     791
FM1     737
SS3     698
FS1     621
SS4     600
SS5     593
EW1     557
FM2     522
SB1     511
EW2     482
SI1     468
SI2     463
SS6     462
F1      452
AD1     446
FS2     413
T1      402
FM3     402
F2      398
Name: Item_ID, dtype: int64

In [213]:
# scenario defination
# 1. Know who the customer is (Customer_ID), with ZIP code
# 2. Do not know who the customer is, only ZIP code
# 3. customer did not provide any information, recommen based on the weather

# number of recommend and number of printed ranking
n_r = 1
n_s = 10

# split the dataset, try different input
train, test = train_test_split(aggregated_table, test_size=0.2, random_state=5)

train_2 = train.drop(['Customer_ID'], axis=1).groupby(['Zip_cd', 'Weather', 'Menu_Category']).sum().reset_index()
test_2 = test.drop(['Customer_ID'], axis=1).groupby(['Zip_cd', 'Weather', 'Menu_Category']).sum().reset_index()

train_3 = train.drop(['Customer_ID', 'Zip_cd'], axis=1).groupby(['Weather', 'Menu_Category']).sum().reset_index()
test_3 = test.drop(['Customer_ID', 'Zip_cd'], axis=1).groupby(['Weather', 'Menu_Category']).sum().reset_index()


test = tc.SFrame(test)
# change the set by different inputs
evaluate_set = train


# data, algo, user_id, item_id, target, n_recom, n_show
# check which secnario of the input:
if 'Customer_ID' in evaluate_set.columns:
    # s1
    evaluate_set = tc.SFrame(evaluate_set)
    m_1_pop, m_1_pop_list = recommender(evaluate_set, 'popularity', 'Customer_ID', 'Menu_Category',
                          'Quantity', n_r, n_s)
    m_1_cos, m_1_pop_list = recommender(evaluate_set, 'cosine', 'Customer_ID', 'Menu_Category',
                          'Quantity', n_r, n_s)
    m_1_per, m_1_pop_list = recommender(evaluate_set, 'pearson', 'Customer_ID', 'Menu_Category',
                          'Quantity', n_r, n_s)
    
##### found there are no output from the below statements #####
elif 'Zip_cd' in evaluate_set.columns:
    # s2
    evaluate_set = tc.SFrame(evaluate_set)
    m_2_pop = recommender(evaluate_set, 'popularity', 'Zip_cd', 'Menu_Category',
                          'Quantity', n_r, n_s)
    m_2_cos = recommender(evaluate_set, 'cosine', 'Zip_cd', 'Menu_Category',
                          'Quantity', n_r, n_s)
    m_2_per = recommender(evaluate_set, 'pearson', 'Zip_cd', 'Menu_Category',
                          'Quantity', n_r, n_s)
elif 'Weather' in evaluate_set.columns:
#     # s3
    evaluate_set = tc.SFrame(evaluate_set)
    m_3_pop = recommender(evaluate_set, 'popularity', 'Weather', 'Menu_Category',
                          'Quantity', n_r, n_s)
    m_3_cos = recommender(evaluate_set, 'cosine', 'Weather', 'Menu_Category',
                          'Quantity', n_r, n_s)
    m_3_per = recommender(evaluate_set, 'pearson', 'Weather', 'Menu_Category',
                          'Quantity', n_r, n_s)

Warning: Ignoring columns Zip_cd, Weather;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 25792 observations with 10683 users and 16 items.

Data prepared in: 0.046801s

...

recommendations finished on 6000/10683 queries. users per second: 1.3245e+06

recommendations finished on 7000/10683 queries. users per second: 1.37094e+06

recommendations finished on 8000/10683 queries. users per second: 1.42806e+06

recommendations finished on 9000/10683 queries. users per second: 1.45584e+06

recommendations finished on 10000/10683 queries. users per second: 1.47667e+06

In [214]:
# the parimary evaluation
eva_model = [m_1_pop, m_1_cos, m_1_per]
eva = tc.recommender.util.compare_models(test, eva_model, model_names=['popularity', 'cosine', 'pearson'])

PROGRESS: Evaluate model popularity


recommendations finished on 1000/4445 queries. users per second: 399361

recommendations finished on 2000/4445 queries. users per second: 528681

recommendations finished on 3000/4445 queries. users per second: 527241

recommendations finished on 4000/4445 queries. users per second: 568343

...

recommendations finished on 1000/4445 queries. users per second: 210793

recommendations finished on 2000/4445 queries. users per second: 187705

recommendations finished on 3000/4445 queries. users per second: 239044

recommendations finished on 4000/4445 queries. users per second: 280859


Precision and recall summary statistics by cutoff
+--------+----------------------+----------------------+
| cutoff |    mean_precision    |     mean_recall      |
+--------+----------------------+----------------------+
|   1    | 0.025871766029246342 | 0.018568571785669667 |
|   2    | 0.042294713160854924 |  0.0637129287410503  |
|   3    | 0.03599550056242971  | 0.08083918081668359  |
|   4    | 0.03695163104611925  | 0.11007677611727104  |
|   5    | 0.03586051743532064  | 0.13405127930437274  |
|   6    | 0.033903262092238436 | 0.15129162426125303  |
|   7    | 0.03276233327976854  | 0.17087917581730866  |
|   8    | 0.036377952755905565 |  0.2207471387505136  |
|   9    | 0.035912385951756276 | 0.24401190922563254  |
|   10   | 0.03794963129608793  | 0.28575633402967476  |
+--------+----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.6614224945597964

Per User RMSE (best)
+-------------+----------------------+-------+
| Customer_ID |         r

In [264]:
# s2
Zip_result = pd.DataFrame()
n_recomm = 3
for i in train_2.Zip_cd.value_counts().index:
    for j in train_2.Weather.value_counts().index:
        temp = train_2[(train_2.Zip_cd==i) & (train_2.Weather==j)].nlargest(3, 'Quantity')
        Zip_result = Zip_result.append(temp)

In [265]:
# Zip_result is the recommendation result of scenario 2
Zip_result

,Zip_cd,Weather,Menu_Category,Quantity
575,70569,rainy,Almond Drink,129
589,70569,rainy,Tofu Pudding,91
582,70569,rainy,Herbal Teas,62
600,70569,sunny,Shaved Ice,281
595,70569,sunny,Fluffy,243
...,...,...,...,...
866,70574,cloudy,Shaved Ice,21
867,70574,cloudy,Signature Series,20
910,70574,windy,Signature Series,39
909,70574,windy,Shaved Ice,24


In [269]:
# s3
W_result = pd.DataFrame()
n_recomm = 3
for j in train_3.Weather.value_counts().index:
    temp = train_3[(train_3.Weather==j)].nlargest(3, 'Quantity')
    W_result = W_result.append(temp)

In [270]:
# W_result is the recommendation result of scenario 3
W_result

,Weather,Menu_Category,Quantity
41,sunny,Shaved Ice,3293
36,sunny,Fluffy,2445
37,sunny,Fresh Milk,1981
30,rainy,Tofu Pudding,1029
16,rainy,Almond Drink,990
26,rainy,Signature Series,828
10,cloudy,Signature Series,922
8,cloudy,Milk Teas,570
2,cloudy,Dessert Combo,541
58,windy,Signature Series,1305
